# 1. Install and Import Dependencies

In [1]:
!pip install torch torchvision torchaudio

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [160]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [161]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [162]:
tokens = tokenizer.encode('Amazing Product, I loved it', return_tensors='pt')

In [163]:
result = model(tokens)

In [164]:
result.logits

tensor([[-2.4945, -2.6093, -1.0464,  1.5463,  3.6674]],
       grad_fn=<AddmmBackward>)

In [165]:
int(torch.argmax(result.logits))+1

5

# 4. Collect Reviews

In [167]:
urls = 300*["https://www.amazon.in/Boat-BassHeads-100-Inspired-Earphones/product-reviews/B07GQD4K6L/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3"]
reviews = []
#ratings = []
for s in range(3,303):
    urls[s-3] = "https://www.amazon.in/Boat-BassHeads-100-Inspired-Earphones/product-reviews/B07GQD4K6L/ref=cm_cr_getr_d_paging_btm_next_%s?ie=UTF8&reviewerType=all_reviews&pageNumber=%s"%(s,s)
    
    r = requests.get(urls[s-3])
    soup = BeautifulSoup(r.text, 'html.parser')
    #regex = re.compile('.*h3YV2d.*')
    results = soup.find_all('span', {'class':"a-size-base review-text review-text-content"})
    #results = soup.find_all('div', {'data-hook':"review"})
    review_list = [result.text.strip("\n") for result in results]
    reviews = reviews + review_list
    
    '''
    results_1 = soup.find_all('span', {'class':"a-icon-alt"})
    #results = soup.find_all('div', {'data-hook':"review"})
    rating_list = [int(result_1.text[0]) for result_1 in results_1]
    ratings = ratings + rating_list 
    #reviews = [review.strip("\n") for review in reviews]
    '''

In [168]:
reviews

['Boat brand was unknown to me. I gave it a shot and bought these earphones after reading several positive reviews. For this price there are no other better earphones out there. First thing I noticed was how strong the connecting cables were, much tougher and sturdier as compared to other well known brands. The earpieces also fit tightly into the ears and seem on the harder side. They will never fall off from the ears. Sound quality is above average as compared to their next product in the hierarchy. Frankly the only missing part is the heavy bass.(this comparison is unfair as I also use Sony XB 550) Other than that an excellent product.',
 "Felt like Cheap Chinese Headphones in some nice packing.Go for Sennheiser, I wasn't able to wear then single day.Pro - So Loud at full they can even bleed your ears too.Cons- High bass driven outshines the highs and the mids ( Vocals lie deep down that boom boom)- Not that comfortable to wear.- even design is So called hawk but it is not if comfort

In [169]:
print(len(reviews), len(ratings))

2020 765


# 5. Load Reviews into DataFrame and Score

In [170]:
import numpy as np
import pandas as pd

In [171]:
data = {"rating" : ratings, "rewiew" : reviews}

In [172]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [173]:
df['review'].iloc[0]

'Boat brand was unknown to me. I gave it a shot and bought these earphones after reading several positive reviews. For this price there are no other better earphones out there. First thing I noticed was how strong the connecting cables were, much tougher and sturdier as compared to other well known brands. The earpieces also fit tightly into the ears and seem on the harder side. They will never fall off from the ears. Sound quality is above average as compared to their next product in the hierarchy. Frankly the only missing part is the heavy bass.(this comparison is unfair as I also use Sony XB 550) Other than that an excellent product.'

In [174]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [176]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [178]:
df

,review,sentiment
0,Boat brand was unknown to me. I gave it a shot...,4
1,Felt like Cheap Chinese Headphones in some nic...,2
2,"Having used SoundMagic ES18s, JBL C200SI and S...",2
3,Okay product. Will comment in 3-6 months regar...,3
4,"honestly saying, i was influenced to buy this ...",1
...,...,...
2015,"I have the same product purchased earlier , wh...",1
2016,Great voice all kind of beats are superb only....,4
2017,Its a Great Product sold by Appario Retail Pri...,5
2018,Using it since 2 years. I don't use them often...,5
